# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [76]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [77]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,21.81,72,40,4.00,MU,1722730583
1,1,edinburgh of the seven seas,-37.0676,-12.3116,12.33,80,97,15.13,SH,1722730828
2,2,amga,60.8953,131.9608,18.41,43,2,1.96,RU,1722730571
3,3,newman,37.3138,-121.0208,41.13,14,16,1.50,US,1722730830
4,4,waitangi,-43.9535,-176.5597,8.79,88,76,1.34,NZ,1722730831


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [78]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City"
    
      
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [79]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Max Temp"] > 21) & (city_data_df["Cloudiness"] < 70) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
ideal_city.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,al bawiti,28.3492,28.8659,27.38,23,0,2.46,EG,1722730875
75,75,tamanrasset,22.7850,5.5228,26.95,29,0,2.57,DZ,1722730884
90,90,kone,-21.0595,164.8658,22.26,43,0,5.04,NC,1722730930
146,146,nova xavantina,-14.6667,-52.3500,24.64,35,0,1.28,BR,1722730995
157,157,aykol,38.1751,77.2132,26.07,43,65,2.86,CN,1722731008
173,173,crane,31.3974,-102.3501,31.37,36,67,8.23,US,1722730790
178,178,whitefish,48.4111,-114.3376,27.86,39,0,0.00,US,1722731033
184,184,baiyin,36.5583,104.2081,21.94,47,0,1.69,CN,1722731040
196,196,mineros,-17.1167,-63.2333,29.26,38,0,2.77,BO,1722731053
207,207,buka,40.8108,69.1986,24.31,28,62,2.74,UZ,1722731067


### Step 3: Create a new DataFrame called `hotel_df`.

In [80]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,al bawiti,EG,28.3492,28.8659,23,None
75,tamanrasset,DZ,22.7850,5.5228,29,None
90,kone,NC,-21.0595,164.8658,43,None
146,nova xavantina,BR,-14.6667,-52.3500,35,None
157,aykol,CN,38.1751,77.2132,43,None
173,crane,US,31.3974,-102.3501,36,None
178,whitefish,US,48.4111,-114.3376,39,None
184,baiyin,CN,36.5583,104.2081,47,None
196,mineros,BO,-17.1167,-63.2333,38,None
207,buka,UZ,40.8108,69.1986,28,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [81]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al bawiti - nearest hotel: Old Oasis Hotel
tamanrasset - nearest hotel: فندق الأمان
kone - nearest hotel: Pacifik Appartels
nova xavantina - nearest hotel: Hotel Morada do Sol
aykol - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
whitefish - nearest hotel: The Firebrand Hotel
baiyin - nearest hotel: 红鹭宾馆
mineros - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
oriximina - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
aleksandrovskoye - nearest hotel: No hotel found
saryozek - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
mwene-ditu - nearest hotel: Hôtel KAMAS
valencia de alcantara - nearest hotel: Hotel El Clavo
trinidad - nearest hotel: Hotel Palace
chu - nearest hotel: No hotel found
hun - nearest hotel: فندق الهروج
yelizovo - nearest hotel: Николина Усадьба
beyneu - nearest hotel: Johan yu
tadine - nearest hotel: Hôtel Nengoné
santa lucia - nearest hotel: Hotel Rural Molino de Aq

,City,Country,Lat,Lng,Humidity,Hotel Name
42,al bawiti,EG,28.3492,28.8659,23,Old Oasis Hotel
75,tamanrasset,DZ,22.7850,5.5228,29,فندق الأمان
90,kone,NC,-21.0595,164.8658,43,Pacifik Appartels
146,nova xavantina,BR,-14.6667,-52.3500,35,Hotel Morada do Sol
157,aykol,CN,38.1751,77.2132,43,No hotel found
173,crane,US,31.3974,-102.3501,36,No hotel found
178,whitefish,US,48.4111,-114.3376,39,The Firebrand Hotel
184,baiyin,CN,36.5583,104.2081,47,红鹭宾馆
196,mineros,BO,-17.1167,-63.2333,38,No hotel found
207,buka,UZ,40.8108,69.1986,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lat","Lng","City","Humidity","Hotel Name","Country"] 
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)